# Importing Dependencies

### the notebook in a nutshell 
- importing 
software dependencies we need
- load_data (please read its description it supposed to be very easy to deal with it)

###  preprocesssing 
- I stored some important variables, then I used log transformation for long tailed distributions

- iterative imputer just like what `MAAB` Advised

- transormed the categorical features to be from the string type just to avoid some fitting isssues cause by the 
OneHotEncoder.

- droped the `participant_ID` after storing it for stratification

- I scaled quantative data and also FMRI data 

### modeling
- base model is logisitc regression with l2 regulization (fancy name for punishing the model to stop overfitting to noise) 
- wrapped inside a multioutput classifier that just gives it the ability to predict 2 targets simultaneously
### validation 
- 5 splits stratifiedKFold works fine no need for grouping as participants are unique 
- regular stratified  = 0.5901173137436236
- RepeatedStratifiedKFold =0.5865925294336686 with 7 minutese delay

In [70]:
import pandas as pd
import numpy as np          
from sklearn.metrics import f1_score  
from sklearn.linear_model import LogisticRegression
from scipy.stats import hmean
from sklearn.multioutput import MultiOutputClassifier
import scipy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer #it might not work directly if not try the following code line

from sklearn.model_selection import StratifiedGroupKFold,StratifiedKFold,RepeatedStratifiedKFold
from pathlib import Path
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")

# Reading Data

In [71]:
"""
just drop the path of your new data directory nothing more than that. 

"""
path = r"C:\Users\Maab\Desktop\ADHD_Kaggle_Competition\Repo\WiDS-Datathon-2025\Data\raw"

def read_data(base_path:str) -> pd.DataFrame :
    path = Path(base_path)
    trc=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_CATEGORICAL_METADATA_new.xlsx')
    trq=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    trf=pd.read_csv(path   /'TRAIN_NEW'  / 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    trs=pd.read_excel(path /'TRAIN_NEW'  / 'TRAINING_SOLUTIONS.xlsx')  
    tsc=pd.read_excel(path /'TEST'      / 'TEST_CATEGORICAL.xlsx')
    tsq=pd.read_excel(path /'TEST'       / 'TEST_QUANTITATIVE_METADATA.xlsx')    
    tsf=pd.read_csv(path   /'TEST'       / 'TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
    sub=pd.read_excel(path / 'SAMPLE_SUBMISSION.xlsx')    
    dic=pd.read_excel(path /'Data Dictionary.xlsx')
    return trc, trq, trf, trs, tsc, tsq, tsf, sub, dic

trc, trq, trf, trs, tsc, tsq, tsf, sub, dic = read_data(base_path=path)

# Data Merging 
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')

# Preprocessing

### Setup

In [72]:
# Highlighting important variables. 
# note that I did't deal with the quantative data trq as categorical
# I will use the OneHotEncoder for the categorical data as we have some data trap that I don't think we can use the label encoder for. 

for feature in trc.columns:
    train[feature] = train[feature].astype(object)
train_ids = train['participant_id']
test_ids = test['participant_id'] # I will store them for later usage in grouping in validation why?  I don't want the same user to appear in both train and test. 
num_feats = [feature for feature in train.columns if train[feature].dtype == 'float64']
cat_feats = [feature for feature in train.columns if train[feature].dtype == 'object'] # seperate categorical and numerical features help me reteriving them later easily for preprocessing.
target_cols = ['ADHD_Outcome', 'Sex_F']
groups = train_ids

### Log Transformation

In [73]:
# Features with positive skewness and non-negative values: apply log transformation
# num_feats[:18] Don't transform fMRI data because it has negative values
log_features = [f for f in num_feats[:18] if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0.5]  # Only apply to features with significant positive skew

# Apply log transformation for the selected features
for feature in log_features:
    train[feature] = np.log1p(train[feature])  # Apply log(x+1) to handle skewed features
    test[feature] = np.log1p(test[feature])    # Apply log(x+1) to test data as well


### Missing Values

In [74]:
# Checking if target columns in train set (adhd and sex) have missing values
train_missing_features_to_impute = train[target_cols].columns[train[target_cols].isnull().any()].tolist()
print(train_missing_features_to_impute) # print column name that has missing values if any.

[]


In [75]:
'''
Train has 25 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan']
Test  has 23 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial']
fMRI has no missing values
Extra columns in train: ['MRI_Track_Age_at_Scan', 'MRI_Track_Scan_Location']
'''

# Find columns with missing values only
train_missing_features_to_impute = train.columns[train.isnull().any()].tolist() # List of features with missing values in train, only 25 and no missing data in fMRI data
test_missing_features_to_impute = test.columns[test.isnull().any()].tolist() # List of features with missing values in test, only 23 and no missing data in fMRI data

# Initialize the imputer
imputer = IterativeImputer(estimator=LassoCV(random_state=42), max_iter=5, random_state=42)

# Impute in-place
train[train_missing_features_to_impute] = imputer.fit_transform(train[train_missing_features_to_impute])
test[test_missing_features_to_impute] = imputer.fit_transform(test[test_missing_features_to_impute])

### Encode Categorical Features

In [76]:
# Convert all categorical features to strings (to avoid mixed types)
for feature in cat_feats:
    train[feature] = train[feature].astype(str)
    test[feature] = test[feature].astype(str)

# One-Hot Encoding for categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

for feature in cat_feats:
    if feature == 'participant_id':  # Skip participant_id since it's not a feature
        continue

    # Apply OneHotEncoder
    train_encoded = encoder.fit_transform(train[[feature]])
    test_encoded = encoder.transform(test[[feature]])

    # Convert encoded features to DataFrame and append them to the original data
    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([feature]))
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([feature]))

    # Drop the original feature and concatenate the new encoded columns
    train = pd.concat([train.drop(columns=[feature]), train_encoded_df], axis=1)
    test = pd.concat([test.drop(columns=[feature]), test_encoded_df], axis=1)

### Drop Irrelevant Columns

In [77]:
for df in (train,test):
    df.drop(columns=['participant_id'], inplace=True) 

### Scale Numerical Features

In [78]:

scaler = StandardScaler()

# Only apply scaling to numerical columns that are not part of the target or categorical features
numerical_features = [col for col in train.columns if col not in target_cols and col not in cat_feats]

# Fit scaler on the numerical features of the train set and transform train and test sets
train[numerical_features] = scaler.fit_transform(train[numerical_features])  # Fit and transform for train set
test[numerical_features] = scaler.transform(test[numerical_features])        # Only transform for test set (avoid data leakage)


# Training and Validation

In [79]:
"""
ayo so there is this modeling thing, the part are 
- base model is a RidgeClassifier with a regulizer (more on that on comments)"
- multioutput classifier is just a wrapper it sucks the base model and elevate its skills to make it output multiple targets.
- there is a pipeline inside the multiooutput classifier that does the following:
1. impute missing values with mean
2. log transform the features that are skewed so bad (you know why now)
3. scale the features to be between 0 and 1
4. PCA to reduce the dimensionality of the features (970 components)
   
"""
features = test.columns 
n_splits = 5 
cv = StratifiedKFold(n_splits=n_splits)
base_model = LogisticRegression(random_state=7,penalty='l2', C=0.02,class_weight="balanced")

model = MultiOutputClassifier(base_model)

# This function will be used to validate the model on one fold at a time, it will called 5 times (5 folds)
def validate(trainset, testset, target_cols):
    """This function takes in:
    trainset: your training data (features + targets)
    testset: your test data (same structure)
    target_cols: list of target column names (because it's a multi-output model)
    """
    weights = ((trainset['Sex_F'] == 1) & (trainset['ADHD_Outcome'] == 1)).astype(int) + 1
    model.fit(trainset.drop(columns=target_cols), trainset[target_cols],sample_weight=weights)
    pred = model.predict(testset.drop(columns=target_cols))
    valid_idx = testset[target_cols].notna().all(axis=1)
    
    valid_testset = testset.loc[valid_idx, target_cols]
    valid_pred = pred[valid_idx]
    f1_scores = [f1_score(valid_testset[col], valid_pred[:, i]) for i, col in enumerate(target_cols)]
    
    # print(f"F1-scores per target: {dict(zip(target_cols, f1_scores))}")
    
    return f1_scores


stds = [] # List to store the standard deviation of each target column in every test split
F1s = [] # List to store the F1 scores returned by your validate() function for each fold

# Cross-validation loop: Stratified K-Fold
"""
# Skf ensures the distribution of the **ADHD_Outcome** target is balanced across the folds. This helps ensure that each fold has a similar proportion of the target classes (e.g., ADHD = 1 and ADHD = 0).
# However, **Sex_F**  is not explicitly balanced during this stratified split. The stratification only considers balancing **ADHD_Outcome**.
# To make SKF balance distribution of Sex_F instead of adhd, uncomment the following line:

for train_index, test_index in cv.split(train.drop(columns=target_cols), train[target_cols[1]]): 
"""
for fold_number, (train_index, test_index) in enumerate(cv.split(train.drop(columns=target_cols), train[target_cols[0]]), 1):
    train_v, test_v = train.iloc[train_index], train.iloc[test_index]
    
    # Get the F1 scores for ADHD and Sex_F
    f1_scores = validate(train_v, test_v, target_cols)
    
    # Print fold number and F1 scores
    print(f"\n==== Fold {fold_number} ===")
    for i, col in enumerate(target_cols):
        print(f"{col} -> F1: {f1_scores[i]:.4f}")
    
    # Append standard deviations of the test set to the stds list
    stds.append(test_v[target_cols].std())
    
    # Append F1 scores to the F1s list
    F1s.append(f1_scores)

F1s = np.array(F1s)

mean_f1_scores = F1s.mean(axis=0)
print("\n==== Overall Results ===")
print("Mean F1-scores:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> F1: {mean_f1_scores[i]:.4f}")
    
    
f1_stds = F1s.std(axis=0)

print("\nF1-score stds:")
for i, target in enumerate(target_cols):
    print(f"    {target} -> Std: {f1_stds[i]:.4f}")
    
print("\nStandard deviations of test sets:")
for std in stds:
    print(f"    {std.to_dict()}")

print("\nscore mean (ADHD Sex): ", np.mean(F1s, axis=0))
score = hmean(F1s, axis=0)
score = hmean(score, axis=0)
print("\nFinal Overall Harmonic Mean (Mimic Leaderboard): ", score)




==== Fold 1 ===
ADHD_Outcome -> F1: 0.7666
Sex_F -> F1: 0.6509

==== Fold 2 ===
ADHD_Outcome -> F1: 0.7190
Sex_F -> F1: 0.5963

==== Fold 3 ===
ADHD_Outcome -> F1: 0.7558
Sex_F -> F1: 0.6383

==== Fold 4 ===
ADHD_Outcome -> F1: 0.7784
Sex_F -> F1: 0.5278

==== Fold 5 ===
ADHD_Outcome -> F1: 0.7529
Sex_F -> F1: 0.6061

==== Overall Results ===
Mean F1-scores:
    ADHD_Outcome -> F1: 0.7546
    Sex_F -> F1: 0.6039

F1-score stds:
    ADHD_Outcome -> Std: 0.0199
    Sex_F -> Std: 0.0430

Standard deviations of test sets:
    {'ADHD_Outcome': 0.46457326315492786, 'Sex_F': 0.4827747459153985}
    {'ADHD_Outcome': 0.46621751673905226, 'Sex_F': 0.46457326315492786}
    {'ADHD_Outcome': 0.46621751673905226, 'Sex_F': 0.48705306873701804}
    {'ADHD_Outcome': 0.46509817553248467, 'Sex_F': 0.46673581783644197}
    {'ADHD_Outcome': 0.46509817553248467, 'Sex_F': 0.47286725722408735}

score mean (ADHD Sex):  [0.75455361 0.60385942]

Final Overall Harmonic Mean (Mimic Leaderboard):  0.66862835472310

# Inference and Submission

In [ ]:
def inference(testdata, model):

    model.fit(train.drop(columns=["ADHD_Outcome", "Sex_F"], axis=1), train[["ADHD_Outcome", "Sex_F"]])
    y_pred = model.predict(test)
    sub['ADHD_Outcome'] = y_pred[:, 0] 
    sub['Sex_F'] = y_pred[:, 1]        
    sub.to_csv(f'../submission{score}.csv', index=False)
    
inference(test, model)